# AE Price Estimation

This notebook presents the *ae_estimation_price* function inside the module *ae_price_estimation* of the package *finance* from *QQuantLib* (**QQuantLib/finance/ae_price_estimation.py**).

This function allows the user to solve an input **option price estimation** problem using **amplitude estimation** algorithms. 

The *ae_estimation_price* functions use the **DensityProbability** and the **PayOff** python classes (explained in notebook **11_ApplicationTo_Finance_02_ClassicalFinance**) for creating the correspondent arrays $p(x)$ and $f(x)$ defined in a domain $x \in [x_0,x_f]$. Then the functions try to compute the expected value

$$\mathbb{E}[f]=\int_{x_0}^{x_f}p(x)f(x)dx$$

using the **q_solve_integral** function explained in notebook **10_ApplicationTo_Finance_01_IntegralComputing**. 

Once the **q_solve_integral** function computes the desired expectation the *ae_estimation_price* functions compute the *option* prices estimation using:

$$V(t, S(t)) = e^{r(T-t)} \int_{x_0}^{x_f}p(x)f(x)dx$$ 


In [ ]:
import sys
sys.path.append("../../")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#This cell loads the QLM solver. QPU = [qlmass, python, c]
from QQuantLib.qpu.get_qpu import get_qpu
QPU = ["qlmass", "python", "c"]
linalg_qpu = get_qpu(QPU[2])

In [ ]:
from QQuantLib.finance.ae_price_estimation import ae_price_estimation

## 1. ae_price_estimation input

The input of the *ae_price_estimation* will be a complete Python dictionary. This Python dictionary needs to have all the mandatory keys for defining an option price estimation problem and solving it with the properly configured **AE** algorithm. 

For pedagogical reasons, we can split this input dictionary in the following sub-dictionaries:

1. Domain configuration.
2. Probability Density configuration 
3. PayOff configuration
4. AE configuration
5. Other configuration


### 1.1 Domain configuration:

This will be the dictionary for configuring the domain and the discretization of our problem. 

So in our case, the domain of the problem will be:

$$x \in [x_0,x_f]$$

and for its discretization:

$$[x_0,x_f]=\{[x_0, x_1]U [x_1, x_2]U ...U [x_{2^n-1}, x_f]\}$$ 

The keys for this *domain* dictionary will be:

* *x0*: the start point of the domain
* *xf*: the end point of the domain
* *n_qbits*: for splitting the domain in $2^{n\_qbits}$ intervals

In [ ]:
#Domain configuration
domain_configuration = {
    'x0': 0.01,
    'xf': 5.0,
    'n_qbits': 5,      
}

### 1.2 Probability Density Configuration 

This dictionary will have the information related to the underlying asset of the option, the information of the market and the time when the probability density will be evaluated (in general this will be the *maturity* of the option). This dictionary will need all the mandatory keys for configuring properly the *DensityProbability* (from **QQuantLib.finance.probability_class**).

* *probability_type*: string with probability density type we want to use. Only 'Black-Scholes' is available (it is expected to implement more density probability functions in the future...)
* *s_0*: Initial value of the asset $S$.
* *risk_free_rate*: the risk-free rate of the market.
* *maturity*: this is the time for computing the probability distribution of the asset value. When we work with option this time will be the *maturity* of the option.
* *volatility*: volatility of the asset up to the initial time.


In [ ]:
#Probability density configuration
probability_configuration = {
    'probability_type': 'Black-Scholes',
    's_0': 1,
    'risk_free_rate': 0.05,
    'maturity': 1.0,
    'volatility': 0.5,    
}

### 1.3 PayOff configuration 

This dictionary will have the mandatory keys for configuring the *PayOff* (from **QQuantLib.finance.payoff_class**). In general will be related to the type of the option, the strike etc...

* *pay_off_type*: this will be the type of option we want. At this moment following options are available:
    * European_Call_Option
    * European_Put_Option
    * Digital_Call_Option
    * Digital_Put_Option
    * Futures
* *strike*: strike of the derivative product.
* *coupon*: this for *Digital_Call_Option* and *Digital_Put_Option*.

In [ ]:
#PayOff Configuration
payoff_configuration = {
    'pay_off_type': 'European_Call_Option',
    'strike': 0.5,
    'coupon': None,        
}

### 1.4 Encoding configuration

This dictionary will have all the mandatory keys to configure the encoding of the $p(x)$ and $f(x)$ arrays into the quantum circuit. These keys will be used for configuring properly the *Encoding* class (from **QQuantLib.DL.encoding_protocols**) used inside the *q_solve_integral* function (from **QQuantLib.finance.quantum_integration**):

* *encoding*: type of encoding used. It can be 0, 1 or 2.
* *multiplexor*: for using multiplexor constructions for controlled by state gates.


In [ ]:
#For encoding class
encoding_configuration = {
    "encoding" : 0, #1,2
    "multiplexor": True,        
}

### 1.5 AE configuration

This dictionary will have all the mandatory keys for configuring the *AE* technique used for solving the integral. These keys will be used for configuring properly the *q_solve_integral* function (from **QQuantLib.finance.quantum_integration**) which needs the *AE* class (from **QQuantLib.AE.ae_class**):

* *ae_type*: **AE** algorithm used. It can be: *MLAE, IQAE, RQAE, CQPEAE, IQPEAE*.
* *schedule*: schedule used for the *MLAE*
* *delta*: parameter for *MLAE*
* *ns*: parameter for *MLAE*
* *auxiliar_qbits_number*: number of auxiliary qubits used in *CQPEAE*
* *cbits_number*: number of classical bits used in *IQPEAE*
* *epsilon*: desired error for the estimation. Used in *IQAE, RQAE*
* *alpha*: confidence for *IQAE* method.
* *gamma*: confidence for *RQAE* method.
* *q*: amplification of the Grover applications used in *RQAE*
* *shots*: number of measurements performed in the quantum circuit: used in *IQAE, CQPEAE, IQPEAE*.
* *mcz_qlm*: if **True** multiplexor version of the multi-controlled Z gate will be used. If **False** QLM default implementation will be used.

In [ ]:
m_k = [0, 1, 2, 3, 4, 5, 6, 7, 8]
ae_configuration = {
    #Amplitude Estimation selection
    'ae_type': 'MLAE', #IQAE, RQAE, CQPEAE, IQPEAE    
    
    #MLAE configuration
    'schedule': [
        m_k,
        [100]*len(m_k)
    ],
    'delta' : 1.0e-7,
    'ns' : 10000,
    
    #CQPEAE configuration
    'auxiliar_qbits_number': 14,
    
    #IQPEAE configuration
    'cbits_number': 10,  
    
    #IQAE & RQAQE
    'epsilon': 0.0001,
    #IQAE
    'alpha': 0.05,
    #RQAE
    'gamma': 0.05,
    'q': 1.2,
    
    #shots
    'shots': 100,      
    #Multi controlled decomposition
    'mcz_qlm': False,
    

        
}

### 1.5 Other configuration

This dictionary will be related to other configuration keys like:

* *qpu*: type of **QPU** used for simulating the **AE** algorithms
* *save*: boolean for saving or not the results
* *file_name*: string with the complete path for saving results
* *number_of_tests*: for doing several repetitions of the solution of a complete give ae problem

In [ ]:
other_configuration = {
    'qpu': linalg_qpu,
    "save": False,
    "file_name": "./ae_problem.csv",
    "number_of_tests": 1
}

In [ ]:
price_estimation_configuration = {}
price_estimation_configuration.update(domain_configuration)
price_estimation_configuration.update(probability_configuration)
price_estimation_configuration.update(payoff_configuration)
price_estimation_configuration.update(encoding_configuration)
price_estimation_configuration.update(ae_configuration)
price_estimation_configuration.update(other_configuration)

In [ ]:
price_estimation_configuration

## 2. ae_price_estimation workflow

The *ae_price_estimation* function will execute the following workflow:

1. Creates the discretized domain using the keys from the *domain_configuration* of the input dictionary.
2. Using the *probability_configuration* of the input dictionary creates a properly configured **DensityProbability** object. 
3. Uses the **DensityProbability** object and the domain discretization for creating the numpy array $p(x)$ for the probability distribution.
4. Using the *payoff_configuration*  of the input dictionary creates a properly configured **PayOff** object. 
5. Uses the **PayOff** object and the domain discretization for creating the numpy array $f(x)$ for the payoff of the selected derivative option.
6. Normalisation of the $p(x)$ and $f(x)$ arrays.
7. Adding the normalised numpy arrays to the input dictionary.
8. Execute the *q_solve_integral*. The input of this function will be the Python dictionary of step 7:
    * 8.1 The *q_solve_integral* function uses the *encoding_configuration* for configuring properly the *Encoding* class.
    * 8.2 The *q_solve_integral* function uses the *ae_configuration* for configuring properly the *AE* class.
    * 8.3 The *q_solve_integral* function executes the *ae* algorithm for computing the estimation of the amplitude and the desired integral. 
9. The *q_solve_integral* will return the desired expected value computed using **AE** integral techniques. 
10. Post-process of the results for providing the price estimation solution.

The output of the *ae_price_estimation* function will be a pandas DataFrame with all the configuration provided information, and the results.


In [ ]:
pdf = ae_price_estimation(**price_estimation_configuration)

In [ ]:
pdf

In [ ]:
pdf.columns

The returned DataFrame has a lot of information. 

For traceability we have include all the keys of the input dictionary:

In [ ]:
#Domain configuration
pdf[['x0', 'xf', 'n_qbits']]

In [ ]:
#Probability configuration
pdf[['probability_type', 's_0', 'risk_free_rate','maturity', 'volatility']]

In [ ]:
#PayOff configuration
pdf[['pay_off_type', 'strike', 'coupon']]

In [ ]:
#ae configuration
pdf[[
    'ae_type','schedule', 'delta', 'ns', 'auxiliar_qbits_number', 'cbits_number',
    'epsilon', 'alpha', 'gamma', 'q', 'shots', 'mcz_qlm', 'encoding','multiplexor']]

In [ ]:
#other configuration
pdf[['qpu', 'save', 'file_name', 'number_of_tests']]

The normalisation of the numpy arrays $p(x)$ and $f(x)$ are included too:

In [ ]:
pdf[['payoff_normalisation', 'p_x_normalisation']]

The columns related to the **AE** integration are:

* *riemman_expectation*: this is the expectation computed as a Riemann, this is the scalar product of the $p(x)$ and the $f(x)$ arrays. 
* *ae_expectation*: expectation computed using **AE** integration techniques.
* *ae_l_expectation*: lower value of the expectation computed using **AE** integration techniques (for **IQAE** and **RQAE**)
* *ae_u_expectation*: upper value of the expectation computed using **AE** integration techniques (for **IQAE** and **RQAE**)
* *absolute_error*: absolute difference between the **AE** integral estimation and the desired integral (*riemann_expectation*)

In [ ]:
pdf[[
    "ae_l_expectation", "ae_expectation", 
    "ae_u_expectation", "riemann_expectation", "absolute_error"]]

In [ ]:
(np.abs(pdf["ae_expectation"] - pdf["riemann_expectation"]), pdf["absolute_error"])

The columns related with the finance quantities allways beginf by *finance* and are:

* *finance_exact_price*: the price of the option computed using an analytical Black-Scholes model
* *finance_riemann_price*: the price of the option when the discount of the risk free ratio is taken into account when using the Riemman aproximation: $\text{riemann_expectation} \; e ^{ r *T}$ with $r$, the *risk free ratio* and $T$ the *maturity*
* *finance_price_estimation*: the price estiamtion of the option using **AE**. This is with the discount of the risk free ratio: $\text{ae_expectation} \; e ^{ r *T}$ with $r$, the *risk free ratio* and $T$ the *maturity*
* *finance_error_riemann* : error between the estimated price option and the Rieman obtained price: $|\text{finance_price_estimation} -\text{finance_riemann_price}|$
* *finance_error_exact*: error between the estimated price option and the exact Rieman obtained price: $|\text{finance_price_estimation} -\text{finance_exact_price}|$

In [ ]:
pdf[['finance_exact_price', 'finance_riemann_price', 
     'finance_price_estimation', 'finance_error_riemann', 'finance_error_exact'
]]

In [ ]:
# The different errors that are delivered
pdf[[
    'absolute_error', 'finance_error_riemann', 'finance_error_exact']]

Finally, other important information is provided:

* *schedule_pdf*: this  is the number of Grover-like operator applications and the number of shots for each application (this is valid for **MLAE**, **IQAE** and **RQAE**)
* *oracle_calls*: total number of calls to the oracle for getting the obtained solution.
* *max_oracle_depth*: maximum number of Grover-like operator applications for getting the obtained solution.
* *run_time*:  simulation time for getting the obtained solution.
* *circuit_stasts*: gate statistics of the different quantum circuits used.


In [ ]:
pdf[['schedule_pdf']].iloc[0].values

In [ ]:
pdf[['oracle_calls', 'max_oracle_depth', 'run_time']]

In [ ]:
pdf['circuit_stasts'].iloc[0]

## 3. Considerations

Several considerations about the *ae_price_estimation* function are summarised in this section.

### 3.1 Encoding considerations.

As explained in notebook *09_DataEncodingClass.ipynb* there are 3 different encodings: 0, 1 y 2. The encoding 0 can be used for encoding strictly defined positive functions $f(x)$. If this condition is not satisfied then encodings 1 and 2 should be used. For pure price estimation, the payoffs will satisfy this condition so encoding 0 is enough. In our case, encoding 0 can be used for:

* European_Call_Option
* European_Put_Option
* Digital_Call_Option
* Digital_Put_Option

In the case of the *Futures* the function can not satisfy the positive defined function $f(x)$ so for this derivative product encodings 1 and 2 should be used!!

In [ ]:
#Future with encoding 0
price_estimation_configuration.update({
    'pay_off_type': 'Futures',
    'strike': 0.8,
    'encoding': 0,
    'ae_type': 'IQAE'
})

In [ ]:
pdf_future = ae_price_estimation(**price_estimation_configuration)

In [ ]:
#Amplitude Estimation Integral computation results
pdf_future[[
    "ae_l_expectation", "ae_expectation", 
    "ae_u_expectation", "riemann_expectation", "absolute_error"]]

In [ ]:
# Finance related prices
pdf_future[['finance_exact_price', 'finance_riemann_price', 
     'finance_price_estimation', 'finance_error_riemann', 'finance_error_exact'
]]

As can be seen in the before cells the solution for our future using encoding 0 is very bad. Encoding 1 or 2 should be used instead!!

In [ ]:
#Future with encoding 2
price_estimation_configuration.update({
    'pay_off_type': 'Futures',
    'strike': 0.8,
    'encoding': 2,
    'ae_type': 'IQAE'
})

In [ ]:
pdf_future = ae_price_estimation(**price_estimation_configuration)

In [ ]:
#Amplitude Estimation Integral computation results
pdf_future[[
    "ae_l_expectation", "ae_expectation", 
    "ae_u_expectation", "riemann_expectation", "absolute_error"]]

In [ ]:
# Finance related prices
pdf_future[['finance_exact_price', 'finance_riemann_price', 
     'finance_price_estimation', 'finance_error_riemann', 'finance_error_exact'
]]

With the encoding 2 the solution of the Future is now well estimated!!!

### 3.2 Encoding 0 and RQAE method.

When using the encoding 0 the **RQAE** technique can not be used. Instead of an error a warning is raised. The *ae_price_estimation* function provides output pandas DataFrame but the columns related with the results will have a None.

In [ ]:
#Future with encoding 0
price_estimation_configuration.update({
    'pay_off_type': 'Futures',
    'strike': 0.8,
    'encoding': 0,
    'ae_type': 'RQAE'
})

In [ ]:
pdf_future = ae_price_estimation(**price_estimation_configuration)

In [ ]:
pdf_future

In [ ]:
#Amplitude Estimation Integral computation results
pdf_future[[
    "ae_l_expectation", "ae_expectation", 
    "ae_u_expectation", "riemann_expectation", "absolute_error"]]

In [ ]:
# Finance related prices
pdf_future[['finance_exact_price', 'finance_riemann_price', 
     'finance_price_estimation', 'finance_error_riemann', 'finance_error_exact'
]]

### 3.3 Negative integrals and RQAE

For pure option price estimation (European_Call_Option, European_Put_Option, Digital_Call_Option, Digital_Put_Option) the price always will be positive so all the **AE** methods can be used (of course the 3 encoding methods can be used). However, there are other financial products where the expected return is needed and it is not necessarily positive (for example de Futures). In these cases, the use of encoding 1 or 2 is not enough for getting the correct results. If the solution is negative the only **AE** technique suitable will be the **RQAE** one.

In [ ]:
#Future with encoding 2
price_estimation_configuration.update({
    'pay_off_type': 'Futures',
    'strike': 1.5,
    'encoding': 2,
    'ae_type': 'IQAE'
})

In [ ]:
pdf_future = ae_price_estimation(**price_estimation_configuration)

In [ ]:
#Amplitude Estimation Integral computation results
pdf_future[[
    "ae_l_expectation", "ae_expectation", 
    "ae_u_expectation", "riemann_expectation", "absolute_error"]]

In [ ]:
# Finance related prices
pdf_future[['finance_exact_price', 'finance_riemann_price', 
     'finance_price_estimation', 'finance_error_riemann', 'finance_error_exact'
]]

As can be seen when the expectation is negative the **AE** method fails despite of the encoding procedure

In [ ]:
#Future with encoding 2
price_estimation_configuration.update({
    'pay_off_type': 'Futures',
    'strike': 1.5,
    'encoding': 2,
    'ae_type': 'RQAE'
})
pdf_future = ae_price_estimation(**price_estimation_configuration)

In [ ]:
#Amplitude Estimation Integral computation results
pdf_future[[
    "ae_l_expectation", "ae_expectation", 
    "ae_u_expectation", "riemann_expectation", "absolute_error"]]

In [ ]:
# Finance related prices
pdf_future[['finance_exact_price', 'finance_riemann_price', 
     'finance_price_estimation', 'finance_error_riemann', 'finance_error_exact'
]]

As can be seen the **RQAE** algorithm give us the correct value!!!

### 3.4 CQPEAE and IQPEAE  techniques

The *ae_price_estimation* can be used with all the encodings and all the **AE** algorithms implemented in the **QQuantLib**. However, a piece of advice should be given when **CQPEAE** and **IQPEAE** algorithms are used. These algorithms used pure **Quantum Phase Estimation** for computing the integrals. These kinds of methods are very computing demand and usually large simulation times are needed for solving the price estimation problem. 